In [42]:
import pandas as pd
import numpy as np
import json, requests
import time
import glob
import re
import copy

from bs4 import BeautifulSoup

In [2]:
def list_to_words(lst):
    if len(lst) == 1:
        return lst[0]
    elif len(lst) == 2:
        return f"{lst[0]} and {lst[1]}"
    else:
        return f"{lst[0]}, {list_to_words(lst[1:])}"
    
df = pd.read_csv("school_country_continent_mapping.csv")
unis = df['Partner University']

In [3]:
location = {}
for i, uni in enumerate(unis):
    page = requests.get(f"https://en.wikipedia.org/wiki/{uni.replace(' ','_')}")
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
        for tag in soup.find_all(class_="locality"):
            links = tag.find_all('a')
            contents = list(map(lambda link: link.contents, links))
            uni_location = []
            for content in contents:
                if len(str(content[0])) < 20:
                    uni_location.append(content[0])
            location[uni] = uni_location
    else:
        location[uni] = []

In [4]:
not_found = set()
multiple_found = {}
for uni in location:
    if location[uni] == []:
        not_found.add(uni)
    elif len(location[uni]) > 1:
        multiple_found[uni] = location[uni]

In [13]:
cities = {}
for uni in location:
    if len(location[uni]) == 1:
        cities[uni] = location[uni][0]

for uni in multiple_found:
    print(uni, location[uni])
    cities[uni] = input()

Aarhus School of Business ['Aarhus', 'Herning']
Aarhus
Chulalongkorn University ['Pathum Wan', 'Bangkok']
Pathum Wan
City University of Hong Kong ['Kowloon Tong', 'Kowloon', 'Hong Kong']
Kowloon
Colgate University ['Hamilton Village', 'Hamilton']
Hamilton Village
Copenhagen Business School ['Frederiksberg', 'Copenhagen']
Frederiksberg
Durham University ['Durham', 'Stockton-on-Tees']
Durham
ESSEC Business School ['Paris', 'France', 'Queenstown', 'Singapore', 'Rabat', 'Morocco']
Paris
Ecole Polytechnique Federale de Lausanne ['Écublens', 'Lausanne']
Lausanne
Hankuk University of Foreign Studies ['Seoul', 'Yongin']
Seoul
Hong Kong University of Science & Technology ['Clear Water Bay', 'Sai Kung']
Sai Kung
Koc University ['Sarıyer', 'Istanbul']
Istanbul
Leiden University ['Leiden', 'The Hague']
Leiden
Linkoping University ['Linköping', 'Norrköping', 'Lidingö']
Linköping
Mahidol University ['Bangkok', 'Bangkok', 'Sai Yok District']
Bangkok
Meiji University ['Chiyoda', 'Suginami', 'Nakano', 

In [16]:
for uni in not_found:
    print(uni)
    cities[uni] = input()

NOC Silicon Valley
San Francisco
Boston College, Massachusetts
Boston
University of Toronto (Mississauga)
Mississauga
CentraleSupelec - Ecole Centrale Paris
Paris
Universita degli Studi di Trento
Trento
Korea Advanced Inst of Sci & Tech
Daejeon
University of Commerce 'Luigi Bocconi' Milan
Milan
California State University East bay
Hayward
Universiteit van Stellenbosch
Stellenbosch
University of Brussels, Solvay Business School
Brussels
Central University of Finance & Economics
Beijing
University College London, University of London
London
University of Kaiserslautern, FRG
Kaiserslautern
Interdisciplinary Center Herzliya (IDC)
Herzliya
Universite Catholique De Louvain
Louvain-la-Neuve
University of Victoria, British Columbia
Greater Victoria, British Columbia
Universidad Autonoma de Madrid
Madrid
Grenoble Institute of Technology (Grenoble INP)
Grenoble
Hautes Etudes Commerciales (HEC) Paris
Paris
Universidad Catolica De Honduras
Honduras
King's College London
London
NOC Shanghai
Shangha

In [27]:
cities_list = []
for uni in unis:
    if uni in cities:
        cities_list.append(cities[uni])
    else:
        cities_list.append(None)

In [30]:
df['City'] = cities_list
df.to_csv('school_country_continent_city_mapping.csv',index=False)

In [54]:
df1 = pd.read_csv("school_country_continent_city_mapping.csv")

In [55]:
j = {}
for i in range(len(df)):
    school = df1.iloc[i]['Partner University']
    city = df1.iloc[i]['City']
    j[school] = city

with open('../src/data/cities.json','w') as f:
    json.dump(j, f)